# Particle Swarm Optimization Code Documentation

Our implementation of PSO relies on two fundamental abstractions (a _solver_ objet which manipulates a collection of _particles_) which are particularly suited to the Object-Oriented Programing (OOP) paradigm.

We therefore implement two python classes which can be imported from the `PSPSO` module (for Parallel Synchronous Particle Swarm Optimization).

## The `Particle` class

### Attributes
We implement a Particle as having the following __attributes__:

- a `position`: the current position of the particle. Initiated uniformly at random in the hypercube $[\texttt{lower}, \texttt{upper}]^{\texttt{ndim}}$ where `lower`, `upper`, and `ndim` are provided by the user during class instantation. Implemented as a `numpy` array.

- a `velocity`: the current velocity of the particle. Initiated as a random requence of -1s and 1s accross `ndim` dimensions. Implemented as a `numpy` array.

- a `personal_best_position` and associated `personal_best_fitness` to keep track of a particle best known position and fitness/error (essentially the particle's memory). Initiated to the particle's original (random) position with an error of positive infinity.

- `c1`, `c2`, `w`: the cognitive, social, and inertia parameters. User-provided, with defaults 1, 2, and .5 respectively.


### Methods

A Particle has the following __methods__:

- `update_velocity(self, global_best)`: given the global best known position, update the particle's velocity according to the PSO rule.

- `move(self)`: move the particle to a new position using the update rule $position_{t+1} = position_{t} + velocity_{t}$. By default, the particles are allowed to explore the search space beyond the initialisation boundaries. To avoid this behaviour, the user should adapt the target function by setting out-of-bound values to positive infinity (the induced discontinuity will not affect the algorithm's performance as it is derivative-free)

## The `PSO` class

A `PSO` object is a solver that implements the PSO algorithm to minimize a given function. It is instantiated as follows:

```python
PSO(num_particles, function, n_iter,
    ndim, lower = -10, upper = 10,
    c1 = 1, c2 = 2, w = .5,
    parallel = False, threadpool = True, epsilon = 10e-7)
```



### Attributes

- `particles`: a `list` of `Particle` objects initiated as described above. There are `num_particles` particles.

- `fitnesses`: an `num_particles`-dimensional array containing the current fitness (aka value) of each of the particles.

- `global_best` and associated `global_best_fitness`: the global best known position along with the function value at that point.

- `function`: the function to be optimised. Has to be written such that it takes a list of positions and returns an array of fitnesses.

- `n_iter`: the number of iterations to run the PSO algorithm for.

- `parallel`: a `True/False` value indicating whether to run the PSO algorithm in Parallel. If `True`, a `pooler` attribute is also created which contains a `Pool` object from the `multiprocessing` module which pools together all your available CPUs at the exception of one.

- `threadpool`: should the parallelization in fact be multi-threading (i.e. different threads with shared memory)? This is typically much more efficient than creating separate processes but isn't really running 'in parallel' because of cPython's Global Interpreter Lock (GIL).

- `epsilon`: defines convergence. If an update between two consecutive global best values is smaller than `epsilon` then the algorithm is said to have converged.

### Methods

- `__init__` and `__str__` to instatiate and print a PSO object.

- `get_fitnesses(self)`: evaluate all particle's fitnesses. It is done in parallel if the `parallel` attribute is set to `True`. Returns a n array of fitnesses.

- `update_particles(self)`: updates each particle's best known position if it's current fitness is better than its previous best.

- `update_best(self)`: get new best position and associated fitness, if applicable.

- `move_particles(self)`: run one iteration of the PSO algorithm. Update particles' velocities and makes them move.

- `run(self, verbose=True)`: runs the algorithm for a prespecified number of steps, or until a convergence criterion is reached. By default, prints the current state of the algorithm every 50 iterations. Returns the algorithm's global best position.

### Example Use

IN:
```python
from PSPSO import Particle, PSO
from functions import quad_function

# Instantiate PSO solver
pso = PSO(num_particles = 20, function = quad_function, n_iter = 100, ndim = 2)

# run the solver
pso.run()
```
----
OUT:


```
Running the PSO algorithm in parallel with 20 particles, for 100 iterations.

Iteration number 0
Current best error: 18.166881777526612

Iteration number 50
Current best error: 3.727553918035805e-12

Found minimum at [2.  0.5] with value 8.0667327796468e-23.
array([2. , 0.5])
```

In [1]:
from PSO import PSO, PSO_synchron, PSO_asynchron
from functions import high_dim_rosenbrock

In [2]:
basic = PSO(20, high_dim_rosenbrock, 200, ndim = 3, epsilon = 10e-12)
basic.run()

Running the PSO algorithm with 20 particles, for at most 200 iterations.

After 155 iterations,
Found minimum at [1.00000303 1.000006   1.00001197] with value 4.557863161978244e-11.


array([1.00000303, 1.000006  , 1.00001197])

In [3]:
basic = PSO_synchron(20, high_dim_rosenbrock, 200, ndim = 3, epsilon = 10e-12)
basic.run()

Running the PSO algorithm with 20 particles, for at most 200 iterations.

After 166 iterations,
Found minimum at [0.99999823 0.99999531 0.99999494] with value 2.029337665174713e-09.


array([0.99999823, 0.99999531, 0.99999494])

In [5]:
asynch = PSO_asynchron(20, high_dim_rosenbrock, 200, ndim = 3, epsilon = 10e-12)
asynch.run()

Running the PSO algorithm asynchronously with 20 particles, for at most 4000 function evaluations.

After 4000 function evaluations,
Found minimum at [1.0000008138131307, 1.0000016227776098, 1.0000032251198856] with value 3.339821618840673e-12.


[1.0000008138131307, 1.0000016227776098, 1.0000032251198856]